In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import numpy_financial as npf

In [2]:
male_df = pd.read_excel("excel_model_credit_life.xlsx", sheet_name=2, usecols="A:L")
female_df = pd.read_excel("excel_model_credit_life.xlsx", sheet_name=2, usecols="N:Y")
new_columns = ["x", "qx", "lx", "dx", "Dx", "Nx", "Cx", "Mx", "Rx", "Sx", "äx", "Ax"]
male_df.columns = new_columns
female_df.columns = new_columns

In [3]:

female_df.dropna(inplace=True)
female_df.drop(2, inplace=True)

In [4]:
male_df.dropna(inplace=True)
male_df.drop(2, inplace=True)

In [5]:
for column in new_columns:
    male_df[column] = pd.to_numeric(male_df[column])
    female_df[column] = pd.to_numeric(female_df[column])

In [6]:
df_index = [ x for x in range(18, 101)]
male_df.index = df_index
female_df.index = df_index
male_df.head()

,x,qx,lx,dx,Dx,Nx,Cx,Mx,Rx,Sx,äx,Ax
18,18,0.000897,100000.000000,89.700000,49362.812101,1.129637e+06,42.575425,5915.245220,274718.888115,2.222786e+07,22.884368,0.119832
19,19,0.001111,99910.300000,111.000343,47421.666979,1.080274e+06,50.659108,5872.669795,268803.642895,2.109823e+07,22.780176,0.123839
20,20,0.001311,99799.299657,130.836882,45547.097603,1.032852e+06,57.415620,5822.010687,262930.973100,2.001795e+07,22.676577,0.127824
21,21,0.001494,99668.462775,148.904683,43737.870537,9.873052e+05,62.831133,5764.595067,257108.962413,1.898510e+07,22.573233,0.131799
22,22,0.001655,99519.558091,164.704869,41992.813614,9.435673e+05,66.825102,5701.763934,251344.367346,1.799780e+07,22.469733,0.135780


In [7]:
net_premium = []
def calculate_premium_rates(loan_amount:int, min_term:int=1, max_term:int=10, min_age:int=18, max_age:int=100, df=male_df, gender:str="male"):
    premium_amount = []
    premium_amount.append({"Gender": gender.upper()})
    
    for age in range(min_age, (max_age + 1)):
        amount_dict = {"age / Term": age}
        for term in range(min_term, (max_term + 1)):
            male_age_n_df,male_age_df =(df.loc[(age + term), new_columns], df.loc[age, new_columns])
            
            Ax_n,aaX_n,Dx_n,Dx,Ax, aax = (male_age_n_df["Ax"], male_age_n_df["äx"], male_age_n_df["Dx"], male_age_df["Dx"], male_age_df["Ax"], male_age_df["äx"])
            
            term_assurance = Ax - ((Dx_n / Dx) * Ax_n)
            temp_assurance = aax - ((Dx_n / Dx) * aaX_n)
            
            initial_expense,renewal_expense, second_year_expense  = (0.55, 0.15, 0.3)
    
            premium = (loan_amount * term_assurance) / (-second_year_expense-initial_expense + renewal_expense + temp_assurance - (temp_assurance * renewal_expense))
            
            amount_dict.update({f"{term}": premium})
           
        premium_amount.append(amount_dict)
    return pd.DataFrame(premium_amount)
    
            
            
    

In [17]:
male_premium_df = calculate_premium_rates(loan_amount=70000000, min_term=10, max_term=10, min_age=30, max_age=30)
female_premium_df = calculate_premium_rates(loan_amount=70000000, min_term=10, max_term=10, min_age=30, max_age=30, df=female_df, gender="Female")
df = pd.concat([male_premium_df, female_premium_df])
df.to_excel("premium_rates.xlsx", index=False, sheet_name="premium_rates")


In [18]:
female_decrement_df = pd.read_excel("excel_model_credit_life.xlsx", sheet_name=5)
male_decrement_df = pd.read_excel("excel_model_credit_life.xlsx", sheet_name=7)
decrement_columns = ["age_x", "qdx", "qbx", "qCx", "aqdx", "aqbx", "aqCx", "apx", "t_1apx"]
female_decrement_df.columns = decrement_columns
male_decrement_df.columns = decrement_columns

male_decrement_df.columns = decrement_columns
female_decrement_df.drop(0, inplace=True)
male_decrement_df.drop(0, inplace=True)
female_decrement_df.index = female_decrement_df.loc[female_decrement_df.index, "age_x"]
male_decrement_df.index = male_decrement_df.loc[male_decrement_df.index, "age_x"]
male_decrement_df.head()

,age_x,qdx,qbx,qCx,aqdx,aqbx,aqCx,apx,t_1apx
age_x,,,,,,,,,
18,18,0.000897,0.000594,0.000964,0.000896,0.000593,0.000963,0.99851,1
19,19,0.001111,0.000594,0.000964,0.00111,0.000593,0.000963,0.998296,0.99851
20,20,0.001311,0.000594,0.000964,0.00131,0.000593,0.000963,0.998097,0.996809
21,21,0.001494,0.000592,0.001006,0.001493,0.000591,0.001005,0.997916,0.994912
22,22,0.001655,0.00059,0.001041,0.001654,0.000589,0.00104,0.997757,0.992839


In [19]:
def calculate_armotization(rate:float, loan_term:int, principal:int):
    loan_schedule = []
    loan_balance = []
    for n in range(0, loan_term + 1):
        
        if n == 0:
           loan_balance.append([principal])
           
        if n > 0:
            pmt = npf.pmt(rate/1, loan_term*1, -principal).item()
            ipmt = npf.ipmt(rate/1, n, loan_term*1, -principal).item()
            ppmt = npf.ppmt(rate/1, n, loan_term*1, -principal).item()
            balance =  loan_balance[0][0] - ppmt if n == 1 else loan_balance[n-1][0] - ppmt
            loan_balance.append([balance])
        
        if n == 0:
            loan_schedule.append({"Period/Time": n, "Payment": None, "Interest": None, "Principal": None, "Balance": principal})
        else:
            loan_schedule.append({"Period/Time": n, "Payment": format(pmt, ".2f"), "Interest": format(ipmt, ".2f"), "Principal":format(ppmt, ".2f"), "Balance": format(balance, ".2f")})
    return pd.DataFrame(loan_schedule)

loan_armotization = calculate_armotization(0.0699, 10, 70000000)


In [20]:
def reserves(loan_amount:int, loan_term:int=10, age:int=30, df=male_df, gender:str="male"):
    reserves_amount = []
    min_age = age
    net_premium = 0
    for age in range(min_age, (age + loan_term + 1)):
        male_age_df, male_ax_n_df = (df.loc[age, new_columns], df.loc[min_age + loan_term, new_columns])
            
        Ax_n,aaX_n,Dx_n,Dx,Ax, aax = (male_ax_n_df["Ax"], male_ax_n_df["äx"], male_ax_n_df["Dx"], male_age_df["Dx"], male_age_df["Ax"], male_age_df["äx"])
        
        term_assurance = Ax - ((Dx_n / Dx) * Ax_n)
        temp_assurance = aax - ((Dx_n / Dx) * aaX_n)
        
        if age == min_age:
            net_premium = (loan_amount * term_assurance) / temp_assurance
            
        reserves= (loan_amount * term_assurance) - (net_premium * temp_assurance)
        
        reserves_amount.append({"x": age, "Ax": Ax, "aaX": aax, "Ax_n": Ax_n, "aaX_n": aaX_n, "Dx": Dx, "Dx_n": Dx_n, "Term Assurance": term_assurance, "Temp Assurance": temp_assurance, "Reserves": reserves})

    return pd.DataFrame(reserves_amount)

In [21]:
reserves_df = reserves(loan_amount=70000000, age=30)
reserves_df.index = reserves_df.loc[reserves_df.index, "x"]
reserves_df.head()

,x,Ax,aaX,Ax_n,aaX_n,Dx,Dx_n,Term Assurance,Temp Assurance,Reserves
x,,,,,,,,,,
30,30,0.171089,21.551674,0.235138,19.886402,30220.644963,20030.576564,0.015237,8.370748,0.000000
31,31,0.176380,21.414128,0.235138,19.886402,29003.508487,20030.576564,0.013987,7.680062,497.692961
32,32,0.181930,21.269830,0.235138,19.886402,27836.675030,20030.576564,0.012730,6.960071,4242.160568
33,33,0.187742,21.118700,0.235138,19.886402,26717.774524,20030.576564,0.011457,6.209670,10737.961848
34,34,0.193815,20.960800,0.235138,19.886402,25644.387933,20030.576564,0.010151,5.427729,18978.053525


In [22]:
interest_rate = 0.115
rist_discount_rate = 0.09
initial_expenses = 0.55
second_year_expenses = 0.3
renewal_expense = 0.15
last_expense = 300000
premium_rate = 0.0025
critical_illness_rate = 0.5

def profit_margin(loan_amount:int, premiun_rate:int = 0.025, loan_term:int=10, p_age:int=18, df=male_df, gender:str="male"):
    premium_amount = []
    npv_profit = 0
    npv_premium = 0
    profit_margin = 0
    age = p_age
    prev_t_1apx = [[1.0]]
    for t in range(1, loan_term + 1):
        
        male_age_n_df,male_age_df =(df.loc[(age + loan_term), new_columns], df.loc[age, new_columns])
            
        Ax_n,aaX_n,Dx_n,Dx,Ax, aax = (male_age_n_df["Ax"], male_age_n_df["äx"], male_age_n_df["Dx"], male_age_df["Dx"], male_age_df["Ax"], male_age_df["äx"])
            
        term_assurance = Ax - ((Dx_n / Dx) * Ax_n)
        temp_assurance = aax - ((Dx_n / Dx) * aaX_n)
            
        initial_expense,renewal_expense, second_year_expenses  = (0.55, 0.15, 0.3)
    
        premium_of_term = (loan_amount * term_assurance) / (-second_year_expenses -initial_expense + renewal_expense + temp_assurance - (temp_assurance * renewal_expense))
        
        premium_of_loan = premiun_rate * loan_amount
        premium = premium_of_term + premium_of_loan
        # print({"premium": premium, "t": t})
        
        if t == 1:
            expenses = initial_expense * premium
        elif t == 2 :
            expenses = premium * 0.3
        else:
            expenses = premium * 0.15
            
        interests = (premium - expenses) * interest_rate
            
        aqdx = male_decrement_df.loc[age, "aqdx"]
        aqbx = male_decrement_df.loc[age, "aqbx"]
        aqCx = male_decrement_df.loc[age, "aqCx"]
        apx = male_decrement_df.loc[age, "apx"]
        
        if t > 1:
            aqdx = male_decrement_df.loc[age + t -1, "aqdx"]
            aqbx = male_decrement_df.loc[age + t -1, "aqbx"]
            aqCx = male_decrement_df.loc[age + t -1, "aqCx"]
            apx = male_decrement_df.loc[age + t -1, "apx"]

        # print(f"aqbx:{aqbx}")
        reserve_apx = male_decrement_df.at[age + t -1, "apx"]
        age_apx = float(male_decrement_df.at[age, "apx"])
        
        t_1apx = float(prev_t_1apx[0][0])
        reserves_value = reserves_df.loc[(age), "Reserves"]
        if t == 1:
            prev_t_1apx.append([float(format(t_1apx * age_apx, ".7f"))])
          
        if t > 1:
            age_apx = male_decrement_df.at[age + (t - 1), "apx"]
            index = t - 1
            if t == 2:
                t_1apx = prev_t_1apx[index][0]
            else:
                t_1apx = prev_t_1apx[index][0] * age_apx
            
            t_1apx = float(format(t_1apx, ".7f"))
  
            reserves_value = reserves_df.loc[(age+t), "Reserves"]
            prev_t_1apx.append([t_1apx])
        
        #print(t_1apx)
        balance = float(loan_armotization.loc[t, ["Balance"]])
        #print(balance)
        outstanding_loan_death_costs = (balance + last_expense) * aqdx
    
        death_benefit_to_beneficiaries = 0 if t == loan_term else loan_amount* aqdx
        
        disability_cost = balance * aqbx
        critical_cost = critical_illness_rate * balance * aqCx
        
        increase_in_provision = apx * reserves_df.loc[(age + 1), "Reserves"] - reserves_df.loc[(age), "Reserves"] if t == 1 else reserve_apx * reserves_value - reserves_df.loc[((age + t) - 1), "Reserves"]
        
        profit = premium - expenses + interests - outstanding_loan_death_costs - death_benefit_to_beneficiaries - disability_cost - critical_cost - increase_in_provision
        
        profit_signature  = profit * t_1apx
        discounted_v_t = (1/(1+rist_discount_rate))**t
       
        discounted_profit = float(format(float(profit_signature * discounted_v_t), ".2f"))
        
        discounted_v_t_1 = float(format(float((1/(1 + rist_discount_rate))**(t - 1)), ".6f"))
       
        discounted_premium = float(format(float(discounted_v_t_1 * premium * t_1apx), ".4f"))
        
        npv_profit += float(format(discounted_profit, ".2f"))
        npv_premium += float(format(discounted_premium, ".2f"))
        
        premium_amount.append({"t": t, "premium": format(premium, ".0f"), "Expenses": format(expenses, ".2f"), "Interests": format(interests, ".2f"), "Outstanding Loan Death Costs": format(outstanding_loan_death_costs, ".2f"), "Death benefit to beneficiaries": format(death_benefit_to_beneficiaries, ".2f"), "Disability Cost": format(disability_cost, ".2f"), "Critical Cost": format(critical_cost, ".2f"), "Increase in Provision": format(increase_in_provision, ".2f"), "Profit": format(profit, ".2f"), "Profit Signature": format(profit_signature, ".2f"), "Discounted vt": format(discounted_v_t, ".6f"), "Discounted Profit": discounted_profit,"Discounted v t-1": discounted_v_t_1, "Discounted Premium": discounted_premium})
    profit_margin = npv_profit / npv_premium
    
    profit_margin = format(profit_margin * 100, ".3f")
    npv_premium = format(npv_premium, ".2f")
    print(f"NPV of profits {npv_profit}")
    print(f"NPV of premiun {npv_premium}")
    print(f"Profit Margin {profit_margin}%")


    return pd.DataFrame(premium_amount)

In [23]:
profit_margin(70000000, 0.0025, 10, 30)

NPV of profits 263616.37
NPV of premiun 2364138.82
Profit Margin 11.151%


,t,premium,Expenses,Interests,Outstanding Loan Death Costs,Death benefit to beneficiaries,Disability Cost,Critical Cost,Increase in Provision,Profit,Profit Signature,Discounted vt,Discounted Profit,Discounted v t-1,Discounted Premium
0,1,341263,187694.70,17660.37,122914.27,131900.09,39352.25,39234.41,496.45,-162668.71,-162668.71,0.917431,-149237.35,1.000000,341263.0973
1,2,341263,102378.93,27471.68,109944.61,128680.38,37254.28,36582.04,3734.01,-49839.48,-49715.37,0.841680,-41844.43,0.917431,312305.6682
2,3,341263,51189.46,33358.47,97279.72,126089.69,35069.98,33605.39,6469.45,24917.87,24794.81,0.772183,19146.14,0.841680,285815.8170
3,4,341263,51189.46,33358.47,85090.28,124614.24,32675.30,31499.76,8193.25,41359.26,41053.44,0.708425,29083.29,0.772183,261569.0080
4,5,341263,51189.46,33358.47,73467.86,124956.38,29818.34,28685.87,8166.07,58337.57,57760.57,0.649931,37540.41,0.708425,239368.1409
5,6,341263,51189.46,33358.47,61935.46,127325.43,26214.35,25128.24,6109.38,76719.23,75763.26,0.596267,45175.15,0.649931,219033.7144
6,7,341263,51189.46,33358.47,49767.92,131722.20,21639.11,20432.86,1943.84,97926.17,96443.93,0.547034,52758.13,0.596267,200403.9295
7,8,341263,51189.46,33358.47,36032.42,137726.24,15841.39,14750.14,-3992.97,123074.86,120866.89,0.501866,60659.02,0.547034,183333.4142
8,9,341263,51189.46,33358.47,19905.98,144981.91,8688.94,8344.04,-11420.21,152931.45,149736.62,0.460428,68942.90,0.501866,167690.4471
9,10,341263,51189.46,33358.47,656.94,0.00,0.00,0.00,-20185.81,342960.98,334728.92,0.422411,141393.11,0.460428,153355.5797


In [38]:
def profit_margin_given_age(loan_amount:int, min_loan_term:int = 1, max_loan_term:int = 10, min_age:int = 30, df=male_df, gender:str="male"):
    premium_amount = []
    #premium_amount.append({"Gender": gender.upper()})
    premium_amount = []
    npv_profit = 0
    npv_premium = 0
    profit_margin = 0

    prev_t_1apx = [[1.0]]
    
    for age in range(min_age, (min_age+max_loan_term)):
        amount_dict = {"age / Term": age}
        for t in range(min_loan_term, max_loan_term + 1):
            
            if age + t > 60:
                continue
            
            
            male_age_n_df,male_age_df =(df.loc[(age + t), new_columns], df.loc[age, new_columns])
                
            Ax_n,aaX_n,Dx_n,Dx,Ax, aax = (male_age_n_df["Ax"], male_age_n_df["äx"], male_age_n_df["Dx"], male_age_df["Dx"], male_age_df["Ax"], male_age_df["äx"])
                
            term_assurance = Ax - ((Dx_n / Dx) * Ax_n)
            temp_assurance = aax - ((Dx_n / Dx) * aaX_n)
                
            initial_expense,renewal_expense, second_year_expenses  = (0.55, 0.15, 0.3)
        
            premium_of_term = (loan_amount * term_assurance) / (-second_year_expenses -initial_expense + renewal_expense + temp_assurance - (temp_assurance * renewal_expense))
            
            premium_of_loan = premium_rate * loan_amount
            premium = premium_of_term + premium_of_loan
            # print({"premium": premium, "t": t})
            
            if t == 1:
                expenses = initial_expense * premium
            elif t == 2 :
                expenses = premium * 0.3
            else:
                expenses = premium * 0.15
                
            interests = (premium - expenses) * interest_rate
                
            aqdx = male_decrement_df.loc[age, "aqdx"]
            aqbx = male_decrement_df.loc[age, "aqbx"]
            aqCx = male_decrement_df.loc[age, "aqCx"]
            apx = male_decrement_df.loc[age, "apx"]
            
            if t > 1:
                aqdx = male_decrement_df.loc[age + t -1, "aqdx"]
                aqbx = male_decrement_df.loc[age + t -1, "aqbx"]
                aqCx = male_decrement_df.loc[age + t -1, "aqCx"]
                apx = male_decrement_df.loc[age + t -1, "apx"]

            # print(f"aqbx:{aqbx}")
            reserve_apx = male_decrement_df.at[age + t -1, "apx"]
            age_apx = float(male_decrement_df.at[age, "apx"])
            
            t_1apx = float(prev_t_1apx[0][0])
            reserves_value = reserves_df.loc[(age), "Reserves"]
            if t == 1:
                prev_t_1apx.append([float(format(t_1apx * age_apx, ".7f"))])
            
            if t > 1:
                age_apx = male_decrement_df.at[age + (t - 1), "apx"]
                index = t - 1
                if t == 2:
                    t_1apx = prev_t_1apx[index][0]
                else:
                    t_1apx = prev_t_1apx[index][0] * age_apx
                
                t_1apx = float(format(t_1apx, ".7f"))
              
                try:
                    reserves_value = reserves_df.loc[age+ (t - 1), "Reserves"]
                except KeyError:
                    pass
                prev_t_1apx.append([t_1apx])
            
            #print(t_1apx)
            balance = float(loan_armotization.loc[(t -1), ["Balance"]])
            #print(balance)
            outstanding_loan_death_costs = (balance + last_expense) * aqdx
        
            death_benefit_to_beneficiaries = 0 if t == max_loan_term else loan_amount* aqdx
            
            disability_cost = balance * aqbx
            critical_cost = critical_illness_rate * balance * aqCx
            
            try:
                increase_in_provision = apx * reserves_df.loc[(age + 1), "Reserves"] - reserves_df.loc[(age), "Reserves"] if t == 1 else reserve_apx * reserves_value - reserves_df.loc[((age + t) - 1), "Reserves"]
            except KeyError:
                pass
            
            profit = premium - expenses + interests - outstanding_loan_death_costs - death_benefit_to_beneficiaries - disability_cost - critical_cost - increase_in_provision
            
            profit_signature  = profit * t_1apx
            discounted_v_t = (1/(1+rist_discount_rate))**t
        
            discounted_profit = profit_signature * discounted_v_t
            discounted_v_t_1 = (1/(1 + rist_discount_rate))**(t - 1)
        
            discounted_premium = discounted_v_t_1 * premium * t_1apx
            
            npv_profit += discounted_profit
            npv_premium += discounted_premium
            profit_margin = npv_profit / npv_premium
            amount_dict.update({f"{t}": round(premium)})
                
        premium_amount.append(amount_dict)
    return pd.DataFrame(premium_amount)
    

In [39]:
profit_margin_given_age(loan_amount=70000000, min_loan_term=1, max_loan_term=10, min_age=30).head(20)

,age / Term,1,2,3,4,5,6,7,8,9,10
0,30,1021282,429514,379484,360435,350599,345044,342017,340669,340518,341263
1,31,1000641,423829,375598,357836,349185,344763,342765,342333,342996,344434
2,32,984038,419842,373573,357229,349830,346499,345444,345837,347188,349198
3,33,974615,418711,374154,359135,352835,350429,350172,351215,353098,355515
4,34,976859,421334,377837,363759,358268,356568,356879,358379,360598,363240
5,35,992115,427921,384525,370913,365931,364654,365308,367045,369401,372091
6,36,1020385,438069,393820,380248,375428,374321,375077,376839,379146,381726
7,37,1058974,451027,405198,391231,386293,385099,385739,387339,389441,391810
8,38,1105641,466246,418109,403405,398066,396555,396889,398172,399976,402069
9,39,1159038,483047,432073,416293,410304,408282,408156,409040,410493,412269
